In [6]:
!kaggle competitions download -c facial-keypoints-detection


  0%|          | 0.00/76.3M [00:00<?, ?B/s]
100%|██████████| 76.3M/76.3M [00:00<00:00, 1.48GB/s]


In [4]:
import zipfile
import os

zip_path = 'facial-keypoints-detection.zip'
extract_path = 'datasets'
os.makedirs(extract_path, exist_ok=True)

if os.path.exists(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print(f"Successfully unzipped {zip_path} to {extract_path}")
else:
    print(f"Error: {zip_path} not found.")

Successfully unzipped facial-keypoints-detection.zip to datasets


In [5]:
test_zip = "datasets/test.zip"
training_zip = "datasets/training.zip"

for zip_file in (test_zip, training_zip):
    if os.path.exists(zip_file):
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
        print(f"Successfully unzipped {zip_file} to {extract_path}")
    else:
        print(f"Error: {zip_file} not found.")

Successfully unzipped datasets/test.zip to datasets
Successfully unzipped datasets/training.zip to datasets


In [6]:
import csv
import pandas as pd
import numpy as np

train_path = "datasets/training.csv"
test_path = "datasets/test.csv"

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

# train_df.head(3)
test_df.head()

,ImageId,Image
0,1,182 183 182 182 180 180 176 169 156 137 124 10...
1,2,76 87 81 72 65 59 64 76 69 42 31 38 49 58 58 4...
2,3,177 176 174 170 169 169 168 166 166 166 161 14...
3,4,176 174 174 175 174 174 176 176 175 171 165 15...
4,5,50 47 44 101 144 149 120 58 48 42 35 35 37 39 ...


In [7]:
#결측치 확인
train_df.isnull().sum()

# 바로 앞 행의 데이터로 결측치 채우기
train_df.fillna(method="ffill", inplace=True)
# test_df.fillna(method="ffill", inplace=True)

train_df.isnull().sum()

left_eye_center_x            0
left_eye_center_y            0
right_eye_center_x           0
right_eye_center_y           0
left_eye_inner_corner_x      0
left_eye_inner_corner_y      0
left_eye_outer_corner_x      0
left_eye_outer_corner_y      0
right_eye_inner_corner_x     0
right_eye_inner_corner_y     0
right_eye_outer_corner_x     0
right_eye_outer_corner_y     0
left_eyebrow_inner_end_x     0
left_eyebrow_inner_end_y     0
left_eyebrow_outer_end_x     0
left_eyebrow_outer_end_y     0
right_eyebrow_inner_end_x    0
right_eyebrow_inner_end_y    0
right_eyebrow_outer_end_x    0
right_eyebrow_outer_end_y    0
nose_tip_x                   0
nose_tip_y                   0
mouth_left_corner_x          0
mouth_left_corner_y          0
mouth_right_corner_x         0
mouth_right_corner_y         0
mouth_center_top_lip_x       0
mouth_center_top_lip_y       0
mouth_center_bottom_lip_x    0
mouth_center_bottom_lip_y    0
Image                        0
dtype: int64

In [8]:
# 이미지로 변환
def num_to_img(data):
    face_images = []

    for i in range(len(data)):
        image_str = data['Image'][i].split(' ')
        image_int = np.array(image_str, dtype=np.uint8).reshape((96, 96))

        face_images.append(image_int)

    X = np.array(face_images)

    print("이미지 데이터 형태", X.shape)

    return X
    

x_train = num_to_img(train_df)
x_test = num_to_img(test_df)

이미지 데이터 형태 (7049, 96, 96)
이미지 데이터 형태 (1783, 96, 96)


In [10]:
# 'Image' 컬럼을 제외하고 나머지 키포인트 컬럼들만 선택합니다.
def get_label(data):
    y_original = data.drop(['Image'], axis=1).values

    scale_factor = 224 / 96
    y_scaled = y_original * scale_factor

    y_normalized = (y_scaled - 112) / 112

    y = y_normalized

    print("레이블 데이터 형태:", y.shape)
    
    return y

y_train = get_label(train_df)
y_test = get_label(test_df)

레이블 데이터 형태: (7049, 30)
레이블 데이터 형태: (1783, 1)


In [16]:
import datasets
from transformers import ViTImageProcessor, ViTForImageClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
import torch

X_train, X_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

model_name = "google/vit-base-patch16-224-in21k"
processor = ViTImageProcessor.from_pretrained(model_name)

train_dataset = datasets.Dataset.from_dict({"pixel_values": X_train, "labels": y_train})
val_dataset = datasets.Dataset.from_dict({"pixel_values": X_val, "labels": y_val})


def transform(example):
    image = np.array(example['pixel_values'])
    image_rgb = np.stack([image] * 3, axis=-1)

    # 프로세서로 이미지 가공 (리사이징, 정규화, 텐서 변환)
    inputs = processor(images=image_rgb, return_tensors="pt")
 
    # 가공된 이미지 텐서로 'pixel_values' 업데이트
    example['pixel_values'] = inputs['pixel_values'].squeeze(0)

    # 레이블을 float 타입의 텐서로 변환
    example['labels'] = torch.tensor(example['labels']).float()

    return example


train_dataset.set_transform(transform)
val_dataset.set_transform(transform)


model = ViTForImageClassification.from_pretrained(
    model_name,
    num_labels=30,
    problem_type="regression",
    ignore_mismatched_sizes=True,
)

training_args = TrainingArguments(
    output_dir="./facial-keypoints-results",
    num_train_epochs=10,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch", # 매 에폭마다 검증 수행
    save_strategy="epoch",       # 매 에폭마다 모델 저장
    load_best_model_at_end=True, # 훈련 종료 후 최적 모델 로드
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

# Trainer: 모델, 설정, 데이터셋을 받아 훈련을 자동화하는 클래스
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# # 훈련 시작
# trainer.train()
# print("훈련 완료!")

ValueError: Found input variables with inconsistent numbers of samples: [7049, 5639]

In [11]:
model.save_pretrained("face_model")
processor.save_pretrained("face_model")

SafetensorError: Error while serializing: IoError(Os { code: 1224, kind: Uncategorized, message: "요청한 작업은, 사용자가 매핑한 구역이 열려 있는 상태인 파일에서 수행할 수 없습니다." })

In [14]:
from transformers import ViTImageProcessor, ViTForImageClassification
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datasets # Assuming 'datasets' from Hugging Face was imported earlier

# Assuming 'x_test', 'y_test', 'transform', 'trainer', and 'train_df' are defined earlier in the script.

# Load the fine-tuned model and processor
model_path = "face_model"
processor = ViTImageProcessor.from_pretrained(model_path)
model = ViTForImageClassification.from_pretrained(model_path)

# Create and transform the test dataset
test_dataset = datasets.Dataset.from_dict({"pixel_values": x_test, "labels": y_test})
test_dataset.set_transform(transform)

# Get predictions from the trainer
predictions = trainer.predict(test_dataset)
predicted_keypoints = predictions.predictions

def visualize_predictions(images, predicted_keypoints, num_samples=5):
    """
    Visualizes images with their predicted facial keypoints.
    """
    plt.figure(figsize=(15, 15))
    for i in range(num_samples):
        ax = plt.subplot(1, num_samples, i + 1)
        keypoints = predicted_keypoints[i] * 112 + 112
        keypoints = keypoints.reshape(-1, 2)
        
        # Display the image (assuming it's a 2D grayscale array)
        plt.imshow(images[i].reshape(224, 224), cmap='gray')
        plt.scatter(keypoints[:, 0], keypoints[:, 1], c='red', marker='x')
        plt.axis('off')
    plt.show()

# Visualize some predictions
visualize_predictions(x_test, predicted_keypoints)

# Prepare data for submission file
id_lookup_table = pd.read_csv('datasets/IdLookupTable.csv')
feature_names = list(id_lookup_table['FeatureName'])
image_ids = list(id_lookup_table['ImageId'])
row_ids = list(id_lookup_table['RowId'])

# Create a mapping from feature name to its column index in the original training data
feature_name_to_index = {name: i for i, name in enumerate(train_df.columns[:-1])}

# Extract the specific predicted keypoints required by the submission format
locations = []
for feature_name, image_id in zip(feature_names, image_ids):
    image_index = image_id - 1
    feature_index = feature_name_to_index[feature_name]
    
    # Denormalize the predicted keypoint coordinate
    # This should match the denormalization used for training/visualization
    location = (predicted_keypoints[image_index, feature_index] * 112) + 112
    locations.append(location)

# Create and save the submission DataFrame
submission_df = pd.DataFrame({'RowId': row_ids, 'Location': locations})
submission_df.to_csv('submission.csv', index=False)

NameError: name 'trainer' is not defined